we start with 3204769 transcripts, all transcripts were assigned to one of the polygons
from xenium we know that they detected 216323 cells, we detected 167780  or 107806 after processing (including filtering of 26272, because polygon was not uniquely mapped to a cell)

In [1]:
import sys
sys.path.append('utils')
import os
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata
import geopandas as gpd

from stardist.models import StarDist2D

from tifffile import imread, imwrite
from csbdeep.utils import normalize
from shapely.geometry import Polygon, Point
from scipy import sparse
from matplotlib.colors import ListedColormap

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


2024-09-02 12:23:22.153814: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
gdf = gpd.read_file('../data/xenium/stardist_all.geojson')
gdf['id'] = [f"ID_{i+1}" for i, _ in enumerate(gdf.index)]
gdf.crs = None
gdf

,id,geometry
0,ID_1,"POLYGON ((2116.705 352.000, 2115.873 353.964, ..."
1,ID_2,"POLYGON ((2988.448 2654.000, 2988.674 2656.521..."
2,ID_3,"POLYGON ((2971.104 3174.000, 2970.065 3175.604..."
3,ID_4,"POLYGON ((3030.417 3494.000, 3029.509 3496.289..."
4,ID_5,"POLYGON ((3022.134 3302.000, 3021.518 3304.291..."
...,...,...
216348,ID_216349,"POLYGON ((27908.171 21580.000, 27907.469 21581..."
216349,ID_216350,"POLYGON ((28906.574 21238.000, 28907.262 21239..."
216350,ID_216351,"POLYGON ((27663.020 21014.000, 27662.251 21015..."
216351,ID_216352,"POLYGON ((28863.908 20810.000, 28864.447 20810..."


In [3]:
gdf['centroid'] = gdf.geometry.centroid

# If you want to extract the x and y coordinates of the centroids
gdf['centroid_x'] = gdf.centroid.x
gdf['centroid_y'] = gdf.centroid.y

In [4]:

import scanpy as sc
adata = sc.read_h5ad('../data/xenium/outs/transcripts_anndata_final.h5ad')
adata

AnnData object with n_obs × n_vars = 3204769 × 541
    obs: 'sample', 'start_x', 'start_y', 'end_x', 'end_y', 'center_x', 'center_y', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'transcript_ids', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    obsm: 'end_x', 'end_y', 'spatial', 'start_x', 'start_y'

In [17]:
import numpy as np
from scipy.stats import norm
import pandas as pd

def assign_transcripts_to_polygons(transcripts_df, gdf, std_dev):
    # Convert centroids to numpy array
    polygon_centroids = np.array([(p.x, p.y) for p in gdf['centroid']])
    
    # Extract transcript coordinates and counts
    transcript_coords = transcripts_df[['center_x', 'center_y']].values
    transcript_counts = transcripts_df['total_counts'].values
    
    # Compute squared differences and distances
    diff = transcript_coords[:, np.newaxis, :] - polygon_centroids[np.newaxis, :, :]
    squared_diff = np.sum(diff ** 2, axis=2)
    distances = np.sqrt(squared_diff)
    
    # Calculate probabilities
    probabilities = norm.pdf(distances, scale=std_dev)
    
    # Weight probabilities by transcript counts
    weighted_probabilities = probabilities * transcript_counts[:, np.newaxis]
    
    # Normalize the weighted probabilities to sum to 1
    row_sums = np.sum(weighted_probabilities, axis=1, keepdims=True)
    normalized_probabilities = weighted_probabilities / row_sums
    
    # Create a cumulative probability array for each transcript
    cumulative_probabilities = np.cumsum(normalized_probabilities, axis=1)
    
    # Generate random numbers for sampling
    random_values = np.random.rand(len(transcripts_df))
    
    # Determine chosen polygon indices
    chosen_polygon_indices = np.argmax(cumulative_probabilities >= random_values[:, np.newaxis], axis=1)
    
    # Assign polygon IDs to transcripts
    transcript_to_polygon_id = gdf['id'].values[chosen_polygon_indices]
    transcripts_df['assigned_polygon_id'] = transcript_to_polygon_id
    
    return transcripts_df

# Parameters
std_dev = 100 / 2

# Example usage
transcripts_with_assignments = assign_transcripts_to_polygons(adata.obs.iloc[:1000], gdf, std_dev)

# Save to CSV
csv_file_path = 'transcripts_with_assignments.csv'
transcripts_with_assignments.to_csv(csv_file_path, index=False)


3361359336.py (39): 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
# whole polygon geometry, not only centroids



In [19]:
import numpy as np
from scipy.stats import norm
import pandas as pd
from multiprocessing import Pool, cpu_count

def assign_transcripts_chunk(transcripts_chunk, polygon_centroids, gdf_ids, std_dev):
    transcript_coords = transcripts_chunk[['center_x', 'center_y']].values
    transcript_counts = transcripts_chunk['total_counts'].values
    
    # Compute squared differences and distances
    diff = transcript_coords[:, np.newaxis, :] - polygon_centroids[np.newaxis, :, :]
    squared_diff = np.sum(diff ** 2, axis=2)
    distances = np.sqrt(squared_diff)
    
    # Calculate probabilities
    probabilities = norm.pdf(distances, scale=std_dev)
    
    # Weight probabilities by transcript counts
    weighted_probabilities = probabilities * transcript_counts[:, np.newaxis]
    
    # Normalize the weighted probabilities to sum to 1
    row_sums = np.sum(weighted_probabilities, axis=1, keepdims=True)
    normalized_probabilities = weighted_probabilities / row_sums
    
    # Create a cumulative probability array for each transcript
    cumulative_probabilities = np.cumsum(normalized_probabilities, axis=1)
    
    # Generate random numbers for sampling
    random_values = np.random.rand(len(transcripts_chunk))
    
    # Determine chosen polygon indices
    chosen_polygon_indices = np.argmax(cumulative_probabilities >= random_values[:, np.newaxis], axis=1)
    
    # Assign polygon IDs to transcripts
    transcripts_chunk['assigned_polygon_id'] = gdf_ids[chosen_polygon_indices]
    
    return transcripts_chunk

def assign_transcripts_to_polygons(transcripts_df, gdf, std_dev):
    # Convert centroids to numpy array
    polygon_centroids = np.array([(p.x, p.y) for p in gdf['centroid']])
    
    # Get the polygon IDs
    gdf_ids = gdf['id'].values
    
    # Split transcripts_df into chunks for parallel processing
    num_chunks = cpu_count()
    chunk_size = len(transcripts_df) // num_chunks
    transcript_chunks = [transcripts_df.iloc[i*chunk_size:(i+1)*chunk_size] for i in range(num_chunks)]
    
    # Use multiprocessing Pool to process chunks in parallel
    with Pool(processes=num_chunks) as pool:
        results = pool.starmap(assign_transcripts_chunk, 
                               [(chunk, polygon_centroids, gdf_ids, std_dev) for chunk in transcript_chunks])
    
    # Combine the results into a single DataFrame
    transcripts_with_assignments = pd.concat(results, ignore_index=True)
    
    return transcripts_with_assignments

if __name__ == "__main__":
    # Parameters
    std_dev = 100 / 2

    # Example usage
    transcripts_with_assignments = assign_transcripts_to_polygons(adata.obs.iloc[:1000], gdf, std_dev)

    # Save to CSV
    csv_file_path = 'transcripts_with_assignments.csv'
    transcripts_with_assignments.to_csv(csv_file_path, index=False)


python(43811) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(43812) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(43814) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(43815) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(43816) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(43817) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(43818) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(43819) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Process SpawnPoolWorker-17:
Traceback (most recent call last):
  File "/Users/lollijagladiseva/anaconda3/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/lollijagladis

KeyboardInterrupt: 